In [2]:
import numpy as np
import pandas as pd
import spacy

In [12]:
data = pd.read_csv('datasets/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [15]:
print(len(documents))
print(documents[:5])

1103663
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [16]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import PorterStemmer
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [24]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [25]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [26]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [27]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [28]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (484, 1), (4022, 1)]

In [29]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4022 ("dampen") appears 1 time.


In [30]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903873663261261),
 (1, 0.38522586675616677),
 (2, 0.49747840165121515),
 (3, 0.5055284652272439)]


In [31]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [32]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"queensland" + 0.021*"adelaid" + 0.017*"market" + 0.017*"south" + 0.012*"share" + 0.011*"farmer" + 0.011*"victoria" + 0.011*"busi" + 0.010*"flood" + 0.010*"water"
Topic: 1 
Words: 0.056*"australia" + 0.035*"trump" + 0.026*"world" + 0.022*"nation" + 0.015*"tasmania" + 0.013*"island" + 0.011*"polit" + 0.010*"hobart" + 0.009*"drum" + 0.009*"leagu"
Topic: 2 
Words: 0.023*"plan" + 0.018*"council" + 0.016*"health" + 0.016*"rural" + 0.014*"indigen" + 0.013*"turnbul" + 0.013*"chang" + 0.013*"commun" + 0.012*"fund" + 0.011*"meet"
Topic: 3 
Words: 0.025*"kill" + 0.022*"attack" + 0.019*"famili" + 0.018*"donald" + 0.015*"fight" + 0.014*"dead" + 0.013*"crash" + 0.012*"million" + 0.012*"vote" + 0.012*"victim"
Topic: 4 
Words: 0.028*"melbourn" + 0.020*"test" + 0.019*"countri" + 0.018*"live" + 0.018*"coast" + 0.014*"gold" + 0.014*"women" + 0.013*"life" + 0.013*"feder" + 0.013*"student"
Topic: 5 
Words: 0.031*"elect" + 0.022*"canberra" + 0.019*"say" + 0.013*"worker" + 0.011*"roya

In [33]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.011*"drum" + 0.011*"turnbul" + 0.008*"john" + 0.007*"farmer" + 0.007*"rain" + 0.006*"tuesday" + 0.006*"bushfir" + 0.006*"dairi" + 0.006*"malcolm" + 0.006*"flood"
Topic: 1 Word: 0.019*"countri" + 0.019*"rural" + 0.017*"hour" + 0.010*"health" + 0.009*"podcast" + 0.007*"indigen" + 0.007*"commun" + 0.007*"fund" + 0.006*"sport" + 0.006*"servic"
Topic: 2 Word: 0.019*"market" + 0.019*"queensland" + 0.011*"share" + 0.009*"price" + 0.008*"australian" + 0.007*"royal" + 0.007*"commiss" + 0.006*"thursday" + 0.006*"cattl" + 0.006*"dollar"
Topic: 3 Word: 0.011*"govern" + 0.008*"elect" + 0.007*"labor" + 0.006*"say" + 0.006*"marriag" + 0.006*"liber" + 0.006*"octob" + 0.005*"coal" + 0.005*"plan" + 0.005*"parti"
Topic: 4 Word: 0.015*"news" + 0.009*"victorian" + 0.009*"david" + 0.008*"juli" + 0.007*"sexual" + 0.007*"inquest" + 0.007*"mark" + 0.006*"explain" + 0.005*"origin" + 0.005*"breakfast"
Topic: 5 Word: 0.013*"australia" + 0.008*"leagu" + 0.008*"live" + 0.008*"world" + 0.007*"final"

In [34]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [35]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8199568390846252	 
Topic: 0.025*"queensland" + 0.021*"adelaid" + 0.017*"market" + 0.017*"south" + 0.012*"share" + 0.011*"farmer" + 0.011*"victoria" + 0.011*"busi" + 0.010*"flood" + 0.010*"water"

Score: 0.020006848499178886	 
Topic: 0.028*"melbourn" + 0.020*"test" + 0.019*"countri" + 0.018*"live" + 0.018*"coast" + 0.014*"gold" + 0.014*"women" + 0.013*"life" + 0.013*"feder" + 0.013*"student"

Score: 0.02000552974641323	 
Topic: 0.023*"plan" + 0.018*"council" + 0.016*"health" + 0.016*"rural" + 0.014*"indigen" + 0.013*"turnbul" + 0.013*"chang" + 0.013*"commun" + 0.012*"fund" + 0.011*"meet"

Score: 0.02000541053712368	 
Topic: 0.025*"kill" + 0.022*"attack" + 0.019*"famili" + 0.018*"donald" + 0.015*"fight" + 0.014*"dead" + 0.013*"crash" + 0.012*"million" + 0.012*"vote" + 0.012*"victim"

Score: 0.02000484988093376	 
Topic: 0.055*"polic" + 0.020*"perth" + 0.018*"die" + 0.017*"interview" + 0.016*"miss" + 0.014*"arrest" + 0.013*"investig" + 0.013*"break" + 0.011*"driver" + 0.010*"sear

In [36]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5881454944610596	 
Topic: 0.011*"drum" + 0.011*"turnbul" + 0.008*"john" + 0.007*"farmer" + 0.007*"rain" + 0.006*"tuesday" + 0.006*"bushfir" + 0.006*"dairi" + 0.006*"malcolm" + 0.006*"flood"

Score: 0.251772940158844	 
Topic: 0.009*"weather" + 0.008*"kill" + 0.008*"hobart" + 0.007*"north" + 0.007*"korea" + 0.007*"michael" + 0.007*"climat" + 0.006*"august" + 0.005*"storm" + 0.005*"syria"

Score: 0.020012913271784782	 
Topic: 0.019*"countri" + 0.019*"rural" + 0.017*"hour" + 0.010*"health" + 0.009*"podcast" + 0.007*"indigen" + 0.007*"commun" + 0.007*"fund" + 0.006*"sport" + 0.006*"servic"

Score: 0.02001027762889862	 
Topic: 0.018*"charg" + 0.016*"murder" + 0.015*"court" + 0.010*"jail" + 0.010*"child" + 0.010*"accus" + 0.009*"polic" + 0.009*"alleg" + 0.009*"assault" + 0.009*"guilti"

Score: 0.020009933039546013	 
Topic: 0.019*"market" + 0.019*"queensland" + 0.011*"share" + 0.009*"price" + 0.008*"australian" + 0.007*"royal" + 0.007*"commiss" + 0.006*"thursday" + 0.006*"cattl" + 0.

In [37]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.35010334849357605	 Topic: 0.023*"plan" + 0.018*"council" + 0.016*"health" + 0.016*"rural" + 0.014*"indigen"
Score: 0.34988531470298767	 Topic: 0.031*"elect" + 0.022*"canberra" + 0.019*"say" + 0.013*"worker" + 0.011*"royal"
Score: 0.1832789033651352	 Topic: 0.025*"queensland" + 0.021*"adelaid" + 0.017*"market" + 0.017*"south" + 0.012*"share"
Score: 0.016683155670762062	 Topic: 0.035*"charg" + 0.033*"court" + 0.025*"murder" + 0.023*"north" + 0.021*"face"
Score: 0.01667608879506588	 Topic: 0.056*"australia" + 0.035*"trump" + 0.026*"world" + 0.022*"nation" + 0.015*"tasmania"
Score: 0.016674645245075226	 Topic: 0.025*"kill" + 0.022*"attack" + 0.019*"famili" + 0.018*"donald" + 0.015*"fight"
Score: 0.016674645245075226	 Topic: 0.028*"melbourn" + 0.020*"test" + 0.019*"countri" + 0.018*"live" + 0.018*"coast"
Score: 0.016674645245075226	 Topic: 0.019*"state" + 0.017*"hour" + 0.016*"rise" + 0.015*"govern" + 0.014*"call"
Score: 0.016674645245075226	 Topic: 0.055*"polic" + 0.020*"perth" + 